In [1]:
import pandas as pd
import numpy as np
import re
import torch
import lightgbm as lgb
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# --- Configuration Block ---
class CFG:
    TRAIN_FILE = 'train.csv'
    TEST_FILE = 'train.csv'
    EMBEDDING_MODEL = 'intfloat/e5-base-v2'
    RANDOM_SEED = 42
    TEST_SIZE = 0.2

In [2]:
# This is the most robust IPQ function we've built
def find_ipq(text):
    text = str(text)
    pattern_before = r'(?:pack of|case of|set of|bundle of|pk)\s*(\d+)'
    match = re.search(pattern_before, text, re.IGNORECASE)
    if match: return int(match.group(1))
    pattern_per = r'(\d+)\s*per\s*case'
    match = re.search(pattern_per, text, re.IGNORECASE)
    if match: return int(match.group(1))
    pattern_after = r'(\d+)\s*[-]?\s*(?:pack|count|pk|ct|pcs|case|pouch|pouches|servings|bottles|cans|bars|rolls|units)'
    match = re.search(pattern_after, text, re.IGNORECASE)
    if match: return int(match.group(1))
    pattern_x = r'\(?x\s*(\d+)\)?'
    match = re.search(pattern_x, text, re.IGNORECASE)
    if match: return int(match.group(1))
    return 1

def parse_full_content_final(text):
    if pd.isna(text):
        text = ""
    else:
        text = str(text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    item_name_match = re.search(r'Item Name:\s*(.*?)(?=\s*Bullet Point|Product Description|Value|$)', text, re.DOTALL)
    item_name = item_name_match.group(1).strip().lower() if item_name_match else None
    bullets = re.findall(r'Bullet Point(?:\s+\d+)?:(.*?)(?=\s*Bullet Point|Product Description|Value|$)', text, re.DOTALL)
    bullet_points_text = ' '.join([b.strip() for b in bullets]).strip()
    desc_match = re.search(r'Product Description:\s*(.*?)(?=\s*Value|$)', text, re.DOTALL)
    product_description_text = desc_match.group(1).strip() if desc_match else ''
    description_parts = [part for part in [bullet_points_text, product_description_text] if part]
    description = ' '.join(description_parts).lower() if description_parts else None
    ipq = find_ipq(text)
    value_match = re.search(r'.*Value:\s*([\d\.]+)', text, re.DOTALL)
    unit_match = re.search(r'.*Unit:\s*(.*)', text, re.DOTALL)
    value = float(value_match.group(1).strip()) if value_match else np.nan
    unit = unit_match.group(1).strip().lower() if unit_match else None
    return {'item_name': item_name, 'description': description, 'ipq': ipq, 'value': value, 'unit': unit}

UNIT_CONVERSION_MAP = {
    'fluid ounce(s)': ('fl oz', 1.0), 'fluid ounce': ('fl oz', 1.0), 'fl. oz.': ('fl oz', 1.0),
    'fl. oz': ('fl oz', 1.0), 'fl oz': ('fl oz', 1.0), 'floz': ('fl oz', 1.0),
    'liter': ('fl oz', 33.814), 'liters': ('fl oz', 33.814), 'ltr': ('fl oz', 33.814),
    'milliliter': ('fl oz', 0.033814), 'millilitre': ('fl oz', 0.033814),
    'mililitro': ('fl oz', 0.033814), 'ml': ('fl oz', 0.033814), 'ounce': ('oz', 1.0),
    'ounces': ('oz', 1.0), 'oz': ('oz', 1.0), 'pound': ('oz', 16.0), 'pounds': ('oz', 16.0),
    'lb': ('oz', 16.0), 'gram': ('oz', 0.035274), 'grams(gm)': ('oz', 0.035274),
    'grams': ('oz', 0.035274), 'gramm': ('oz', 0.035274), 'gr': ('oz', 0.035274),
    'kg': ('oz', 35.274), 'count': ('ct', 1.0), 'ct': ('ct', 1.0), 'each': ('ct', 1.0),
    'piece': ('ct', 1.0), 'packs': ('ct', 1.0), 'pack': ('ct', 1.0),
    'bottle': ('ct', 1.0), 'bottles': ('ct', 1.0), 'bag': ('ct', 1.0),
    'bags': ('ct', 1.0), 'can': ('ct', 1.0), 'jar': ('ct', 1.0), 'k-cups': ('ct', 1.0),
    'per carton': ('ct', 1.0), 'pouch': ('ct', 1.0), 'per package': ('ct', 1.0),
    'per box': ('ct', 1.0), 'paper cupcake liners': ('ct', 1.0), 'capsule': ('ct', 1.0),
    'carton': ('ct', 1.0), 'ziplock bags': ('ct', 1.0), 'units': ('ct', 1.0),
    'box': ('ct', 1.0), 'bucket': ('ct', 1.0), 'none': ('None', 1.0), '---': ('None', 1.0)
}

unit_keys = '|'.join(re.escape(k) for k in UNIT_CONVERSION_MAP.keys())
VALUE_IN_TITLE_REGEX = re.compile(r'(\d+(?:\.\d*)?|\.\d+)\s*(' + unit_keys + r')\b|(' + unit_keys + r')\s*(\d+(?:\.\d*)?|\.\d+)\b', re.IGNORECASE)

def extract_and_standardize_from_title(title):
    if pd.isna(title): return np.nan, None
    match = VALUE_IN_TITLE_REGEX.search(str(title))
    if not match: return np.nan, None
    if match.group(1): value_str, unit_str = match.group(1), match.group(2)
    else: unit_str, value_str = match.group(3), match.group(4)
    value = float(value_str)
    unit_lower = unit_str.lower()
    standard_unit, conversion_factor = UNIT_CONVERSION_MAP.get(unit_lower, (None, 1.0))
    if standard_unit: return value * conversion_factor, standard_unit
    return np.nan, None

def standardize_original_value(value, unit):
    if pd.isna(value) or pd.isna(unit): return np.nan, None
    unit_lower = str(unit).lower()
    if unit_lower in UNIT_CONVERSION_MAP:
        standard_unit, conversion_factor = UNIT_CONVERSION_MAP[unit_lower]
        return value * conversion_factor, standard_unit
    for keyword, (mapping, conversion) in UNIT_CONVERSION_MAP.items():
        if keyword in unit_lower: return value * conversion, mapping
    return value, 'other'

def rectify_value_and_unit(row):
    value_from_title, unit_from_title = extract_and_standardize_from_title(row['item_name'])
    value_from_content, unit_from_content = standardize_original_value(row['value'], row['unit'])
    ipq = row['ipq'] if row['ipq'] > 0 else 1
    if pd.notna(value_from_title):
        final_value = value_from_title
        final_unit = unit_from_title
    else:
        final_value = value_from_content / ipq if pd.notna(value_from_content) else np.nan
        final_unit = unit_from_content
    return pd.Series([final_value, final_unit])

def extract_brand_by_length(text):
    if not isinstance(text, str) or not text: return "Unknown"
    words = text.split()
    if len(words) == 0: return "Unknown"
    if len(words) == 1: return words[0]
    first_word = words[0]
    if len(first_word) > 2: return first_word
    else: return ' '.join(words[:2])

In [6]:
def load_and_parse_data(file_path):
    """Loads data and applies the initial parsing function."""
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    df["price"] = pd.to_numeric(df["price"], errors='coerce')
    
    print("Parsing catalog_content...")
    extracted_features = df['catalog_content'].progress_apply(parse_full_content_final).apply(pd.Series)
    df = pd.concat([df, extracted_features], axis=1)
    
    return df

def create_base_features(df):
    """Creates brand, value, and unit features."""
    print("Creating base features (brand, value, unit)...")
    df[['value_final', 'unit_final']] = df.progress_apply(rectify_value_and_unit, axis=1)
    
    # Apply manual rules
    df['value_final'] = np.where((df['unit_final'] == 'ct') & (df['ipq'] == df['value']), 1.0, df['value_final'])
    df['value_final'] = np.where(df['value_final'] > 1000, df['value'], df['value_final'])
    df['ipq'] = np.where(df['ipq'] > 5000, 5000, df['ipq'])
    
    df["value"] = df["value_final"].combine_first(df["value"])
    df["unit"] = df["unit_final"]
    df = df.drop(columns=["value_final", "unit_final"])
    
    df['total_value'] = df['ipq'] * df['value']
    df['brand'] = df['item_name'].progress_apply(extract_brand_by_length)
    
    # Target variable
    df['log_price'] = np.log1p(df['price'])
    
    return df

def generate_text_embeddings(df, text_column, prefix):
    """Generates and adds text embeddings for a specified column."""
    print(f"Generating embeddings for '{text_column}'...")
    
    if torch.cuda.is_available(): device = 'cuda'
    else: device = 'cpu'
    
    model = SentenceTransformer(CFG.EMBEDDING_MODEL, device=device)
    
    texts = df[text_column].fillna('').tolist()
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=128)
    
    emb_df = pd.DataFrame(embeddings, columns=[f'{prefix}_{i}' for i in range(embeddings.shape[1])])
    
    return pd.concat([df.reset_index(drop=True), emb_df.reset_index(drop=True)], axis=1)

def finalize_features(df):
    """Prepares the final feature set for modeling."""
    print("Finalizing features for modeling...")
    
    # Create categorical codes
    df['brand_cat'] = pd.factorize(df['brand'])[0]
    df['unit_cat'] = pd.factorize(df['unit'])[0]
    
    # Drop original text columns
    df = df.drop(columns=['catalog_content', 'item_name', 'description', 'brand', 'unit', 'image_link', 'price'])
    
    return df

In [8]:
def smape(y_true, y_pred):
    """Calculates SMAPE, robust to zero values."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / (denominator + 1e-8)) * 100

def train_and_evaluate_lgbm(df):
    """Trains a LightGBM model and evaluates it using SMAPE."""
    print("\n--- Training LightGBM Model ---")
    
    y = df['log_price']
    X = df.drop(columns=['log_price', 'sample_id'])
    
    categorical_features = ['brand_cat', 'unit_cat']

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=CFG.TEST_SIZE, random_state=CFG.RANDOM_SEED
    )
    
    lgbm = lgb.LGBMRegressor(random_state=CFG.RANDOM_SEED, device='cpu') # Use 'gpu' for NVIDIA
    
    lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=categorical_features,
        callbacks=[lgb.early_stopping(15, verbose=False)]
    )
    
    y_pred_log = lgbm.predict(X_val)
    y_pred_actual = np.expm1(y_pred_log)
    y_true_actual = np.expm1(y_val)
    
    smape_score = smape(y_true_actual, y_pred_actual)
    print(f"✅ Validation SMAPE Score: {smape_score:.4f}%")
    
    return lgbm

In [17]:
from sklearn.linear_model import Ridge

def train_and_evaluate_ridge(df, alpha=1.0):
    """
    Trains a Ridge Regression model and evaluates it using SMAPE.
    Includes a step to handle missing NaN values.
    """
    print("\n--- Training Ridge Regression Model ---")

    y = df['log_price']
    X = df.drop(columns=['log_price', 'sample_id'], errors='ignore')

    # --- FIX: Handle missing values before training ---
    # We will fill any NaN in the feature set with the median of its column.
    # This is a robust way to handle missing data for models like Ridge.
    print("Handling missing values by filling with column median...")
    X = X.fillna(X.median())

    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=CFG.TEST_SIZE,
        random_state=CFG.RANDOM_SEED
    )

    # Initialize the Ridge model
    ridge = Ridge(alpha=alpha, random_state=CFG.RANDOM_SEED)

    # Train the model
    print("Fitting Ridge model...")
    ridge.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred_log = ridge.predict(X_val)
    y_pred_actual = np.expm1(y_pred_log)
    y_true_actual = np.expm1(y_val)

    # Calculate and print the SMAPE score
    smape_score = smape(y_true_actual, y_pred_actual)
    print(f"✅ Validation SMAPE Score: {smape_score:.4f}%")

    return ridge

In [14]:
# --- Run the Full Pipeline ---
tqdm.pandas()

# 1. Load and Parse
train_df = load_and_parse_data(CFG.TRAIN_FILE)

# 2. Create Base Features
train_df = create_base_features(train_df)

# 3. Generate Embeddings (Modular)
train_df = generate_text_embeddings(train_df, text_column='item_name', prefix='name_emb')
train_df = generate_text_embeddings(train_df, text_column='description', prefix='desc_emb')

# 4. Finalize Features
train_final = finalize_features(train_df)

# 5. Train and Evaluate Model
model = train_and_evaluate_lgbm(train_final)

Loading data from train.csv...
Parsing catalog_content...


  0%|          | 0/75000 [00:00<?, ?it/s]

Creating base features (brand, value, unit)...


  0%|          | 0/75000 [00:00<?, ?it/s]

  0%|          | 0/75000 [00:00<?, ?it/s]

Generating embeddings for 'item_name'...


Batches:   0%|          | 0/586 [00:00<?, ?it/s]

Generating embeddings for 'description'...


Batches:   0%|          | 0/586 [00:00<?, ?it/s]

Finalizing features for modeling...

--- Training LightGBM Model ---
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.136679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 395780
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1541
[LightGBM] [Info] Start training from score 2.740904
✅ Validation SMAPE Score: 54.3818%


In [13]:
# --- Run the Full Pipeline ---
tqdm.pandas()

# 1. Load and Parse
train_df = load_and_parse_data(CFG.TRAIN_FILE)

# 2. Create Base Features
train_df = create_base_features(train_df)

# 3. Generate Embeddings (Modular)
train_df = generate_text_embeddings(train_df, text_column='item_name', prefix='name_emb')

# 4. Finalize Features
train_final = finalize_features(train_df)

# 5. Train and Evaluate Model
model = train_and_evaluate_lgbm(train_final)

Loading data from train.csv...
Parsing catalog_content...


  0%|          | 0/75000 [00:00<?, ?it/s]

Creating base features (brand, value, unit)...


  0%|          | 0/75000 [00:00<?, ?it/s]

  0%|          | 0/75000 [00:00<?, ?it/s]

Generating embeddings for 'item_name'...


Batches:   0%|          | 0/586 [00:00<?, ?it/s]

Finalizing features for modeling...

--- Training LightGBM Model ---
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.150680 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 199940
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 773
[LightGBM] [Info] Start training from score 2.740904
✅ Validation SMAPE Score: 54.3609%


In [19]:
# Run the Ridge regression training and evaluation
ridge_model = train_and_evaluate_ridge(train_final, alpha=1.0)


--- Training Ridge Regression Model ---
Handling missing values by filling with column median...
Fitting Ridge model...
✅ Validation SMAPE Score: 59.5602%


In [22]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# --- 1. Modified Training Function to Return Predictions ---
def train_and_get_predictions_lgbm(df):
    """
    Trains a LightGBM model and returns the model, validation predictions,
    and true validation labels.
    """
    print("\n--- Training LightGBM Model ---")
    
    y = df['log_price']
    X = df.drop(columns=['log_price', 'sample_id'], errors='ignore')
    
    categorical_features = ['brand_cat', 'unit_cat']

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=CFG.TEST_SIZE, random_state=CFG.RANDOM_SEED
    )
    
    lgbm = lgb.LGBMRegressor(random_state=CFG.RANDOM_SEED, device='cpu')
    
    lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=categorical_features,
        callbacks=[lgb.early_stopping(15, verbose=False)]
    )
    
    y_pred_log = lgbm.predict(X_val)
    
    return lgbm, y_pred_log, y_val

# --- 2. Run Training and Get Predictions ---
model, y_pred_log, y_val = train_and_get_predictions_lgbm(train_final)

# --- 3. Calculate and Print Overall SMAPE ---
y_pred_actual = np.expm1(y_pred_log)
y_true_actual = np.expm1(y_val)

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / (denominator + 1e-8)) * 100

overall_smape_score = smape(y_true_actual, y_pred_actual)
print("\n--- Model Evaluation Complete ---")
print(f"✅ Overall Validation SMAPE Score: {overall_smape_score:.4f}%")


# --- 4. Perform Detailed Validation Set Analysis ---
train_set, val_set = train_test_split(
    train_df,
    test_size=CFG.TEST_SIZE,
    random_state=CFG.RANDOM_SEED
)

val_results_df = val_set.copy()
val_results_df['predicted_price'] = y_pred_actual

def calculate_smape_per_row(row):
    true = row['price']
    pred = row['predicted_price']
    numerator = np.abs(pred - true)
    denominator = (np.abs(true) + np.abs(pred)) / 2
    if denominator == 0:
        return 0
    return (numerator / denominator) * 100

val_results_df['smape_score'] = val_results_df.apply(calculate_smape_per_row, axis=1)

# --- 5. Display Detailed Results ---
print("\n--- Validation Set with Predicted Prices and SMAPE Scores ---")
print(val_results_df[[
    'sample_id', 'item_name', 'price', 'predicted_price', 'smape_score'
]].head())

print("\n--- Top 5 Worst Performing Rows in Validation Set ---")
print(val_results_df.sort_values(by='smape_score', ascending=False).head(5)[[
    'item_name', 'price', 'predicted_price', 'smape_score'
]])


--- Training LightGBM Model ---
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.141628 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 395780
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1541
[LightGBM] [Info] Start training from score 2.740904

--- Model Evaluation Complete ---
✅ Overall Validation SMAPE Score: 54.3818%

--- Validation Set with Predicted Prices and SMAPE Scores ---
       sample_id                                          item_name   price  \
26837     158784  log cabin sugar free syrup, 24 fl oz (pack of 12)  12.195   
2592        4095  raspberry ginseng oolong tea (50 tea bags, zin...  38.540   
18359     172021  walden farms honey dijon dressing - calorie-fr...  17.860   
73292     268276  vlasic ovals hamburger dill pickle chips, keto...   2.940 

In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# --- 1. Modified Training Function to Return Predictions ---
def train_and_get_predictions_lgbm(df):
    """
    Trains a LightGBM model and returns the model, validation predictions,
    and true validation labels.
    """
    print("\n--- Training LightGBM Model ---")
    
    y = df['log_price']
    X = df.drop(columns=['log_price', 'sample_id'], errors='ignore')
    
    categorical_features = ['brand_cat', 'unit_cat']

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=CFG.TEST_SIZE, random_state=CFG.RANDOM_SEED
    )
    
    lgbm = lgb.LGBMRegressor(random_state=CFG.RANDOM_SEED, device='cpu')
    
    lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=categorical_features,
        callbacks=[lgb.early_stopping(15, verbose=False)]
    )
    
    y_pred_log = lgbm.predict(X_val)
    
    return lgbm, y_pred_log, y_val

# --- 2. Run Training and Get Predictions ---
model, y_pred_log, y_val = train_and_get_predictions_lgbm(train_final)

# --- 3. Calculate and Print Overall SMAPE ---
y_pred_actual = np.expm1(y_pred_log)
y_true_actual = np.expm1(y_val)

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / (denominator + 1e-8)) * 100

overall_smape_score = smape(y_true_actual, y_pred_actual)
print("\n--- Model Evaluation Complete ---")
print(f"✅ Overall Validation SMAPE Score: {overall_smape_score:.4f}%")


# --- 4. Perform Detailed Validation Set Analysis ---
train_set, val_set = train_test_split(
    train_df,
    test_size=CFG.TEST_SIZE,
    random_state=CFG.RANDOM_SEED
)

val_results_df = val_set.copy()
val_results_df['predicted_price'] = y_pred_actual

def calculate_smape_per_row(row):
    true = row['price']
    pred = row['predicted_price']
    numerator = np.abs(pred - true)
    denominator = (np.abs(true) + np.abs(pred)) / 2
    if denominator == 0:
        return 0
    return (numerator / denominator) * 100

val_results_df['smape_score'] = val_results_df.apply(calculate_smape_per_row, axis=1)

# --- 5. Display Detailed Results ---
print("\n--- Validation Set with Predicted Prices and SMAPE Scores ---")
print(val_results_df[[
    'sample_id', 'item_name', 'price', 'predicted_price', 'smape_score'
]].head())

print("\n--- Top 5 Worst Performing Rows in Validation Set ---")
print(val_results_df.sort_values(by='smape_score', ascending=False).head(5)[[
    'item_name', 'price', 'predicted_price', 'smape_score'
]])

# --- NEW: Step 6. Save the Top 1000 Worst Performing Rows ---
print("\n--- Saving Top 1000 Worst Performing Samples ---")

# Sort the entire validation results DataFrame by SMAPE score in descending order
worst_samples_df = val_results_df.sort_values(by='smape_score', ascending=False)

# Select the top 1000 rows
top_1000_worst = worst_samples_df.head(1000)

# Define the columns you want to save for analysis
columns_to_save = [
    'sample_id', 
    'item_name', 
    'price', 
    'predicted_price', 
    'smape_score'
]

# Save the selected columns of the top 1000 worst samples to a CSV file
top_1000_worst[columns_to_save].to_csv('worst_performing_samples.csv', index=False)

print("✅ Successfully saved the top 1000 worst predictions to 'worst_performing_samples.csv'")


--- Training LightGBM Model ---
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.136017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 395780
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 1541
[LightGBM] [Info] Start training from score 2.740904

--- Model Evaluation Complete ---
✅ Overall Validation SMAPE Score: 54.3818%

--- Validation Set with Predicted Prices and SMAPE Scores ---
       sample_id                                          item_name   price  \
26837     158784  log cabin sugar free syrup, 24 fl oz (pack of 12)  12.195   
2592        4095  raspberry ginseng oolong tea (50 tea bags, zin...  38.540   
18359     172021  walden farms honey dijon dressing - calorie-fr...  17.860   
73292     268276  vlasic ovals hamburger dill pickle chips, keto...   2.940 